In [1]:
from influxdb_client import InfluxDBClient
import os

# 🔐 Replace with your actual credentials or load from environment
INFLUX_URL = os.getenv("INFLUX_URL", "http://localhost:8086")
INFLUX_TOKEN = os.getenv("INFLUX_TOKEN", "sentry123")
INFLUX_ORG = os.getenv("INFLUX_ORG", "Sentry")
INFLUX_BUCKET = os.getenv("INFLUX_BUCKET", "Sentry")

client = InfluxDBClient(url=INFLUX_URL, token=INFLUX_TOKEN, org=INFLUX_ORG)
query_api = client.query_api()

# 📦 Query to list all unique symbols from any measurement
flux_query = f'''
import "influxdata/influxdb/schema"

schema.tagValues(
  bucket: "{INFLUX_BUCKET}",
  tag: "symbol"
)
'''

result = query_api.query_data_frame(flux_query)

# ✅ Display symbols
symbols = result["_value"].dropna().unique().tolist()
print("Symbols:", symbols)


c:\Users\srb73\AppData\Local\Programs\Python\Python310\lib\site-packages\influxdb_client\client\warnings.py:31: MissingPivotFunction: The query doesn't contains the pivot() function.

The result will not be shaped to optimal processing by pandas.DataFrame. Use the pivot() function by:

    
import "influxdata/influxdb/schema"

schema.tagValues(
  bucket: "Sentry",
  tag: "symbol"
)
 |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")

You can disable this warning by:
    import warnings
    from influxdb_client.client.warnings import MissingPivotFunction

    warnings.simplefilter("ignore", MissingPivotFunction)

For more info see:
    - https://docs.influxdata.com/resources/videos/pivots-in-flux/
    - https://docs.influxdata.com/flux/latest/stdlib/universe/pivot/
    - https://docs.influxdata.com/flux/latest/stdlib/influxdata/influxdb/schema/fieldsascols/

  warnings.warn(message, MissingPivotFunction)


Symbols: ['1INCHBTC', '1INCHUSDT', 'AAVEBTC', 'AAVEETH', 'AAVEUSDT', 'ACABTC', 'ACATRY', 'ACAUSDT', 'ACHBTC', 'ACHTRY', 'ACHUSDT', 'ACMUSDT', 'ADABNB', 'ADABRL', 'ADABTC', 'ADAETH', 'ADAEUR', 'ADAFDUSD', 'ADATRY', 'ADAUSDC', 'ADAUSDT', 'ADXBTC', 'ADXETH', 'ADXUSDT', 'AGLDBTC', 'AGLDUSDT', 'ALCXUSDT', 'ALGOBTC', 'ALGOTRY', 'ALGOUSDC', 'ALGOUSDT', 'ALICETRY', 'ALICEUSDT', 'ALPACAUSDT', 'ALPHAUSDT', 'ALPINETRY', 'ALPINEUSDT', 'AMPTRY', 'AMPUSDT', 'ANKRBTC', 'ANKRTRY', 'ANKRUSDT', 'APEBTC', 'APETRY', 'APEUSDT', 'API3BTC', 'API3TRY', 'API3USDT', 'APTBTC', 'APTETH', 'APTEUR', 'APTTRY', 'APTUSDT', 'ARBBTC', 'ARBETH', 'ARBEUR', 'ARBFDUSD', 'ARBTC', 'ARBTRY', 'ARBTUSD', 'ARBUSDT', 'ARDRBTC', 'ARDRUSDT', 'ARKMBNB', 'ARKMBTC', 'ARKMTRY', 'ARKMUSDT', 'ARKTRY', 'ARKUSDT', 'ARPABTC', 'ARPATRY', 'ARPAUSDT', 'ARUSDT', 'ASRTRY', 'ASRUSDT', 'ASTRBTC', 'ASTRUSDT', 'ATABTC', 'ATAUSDT', 'ATMTRY', 'ATMUSDT', 'ATOMBTC', 'ATOMETH', 'ATOMEUR', 'ATOMFDUSD', 'ATOMTRY', 'ATOMUSDC', 'ATOMUSDT', 'AUCTIONBTC', 'AUCT

In [2]:
import pandas as pd

pd.DataFrame(symbols, columns=["symbol"]).to_csv("symbols.csv", index=False)
print("✅ Symbols saved to symbols.csv")


✅ Symbols saved to symbols.csv


In [17]:
import requests

COMMON_BUZZWORDS = ["ETF", "SEC", "DeFi", "halving", "bullish", "bearish"]

def generate_keywords_for_symbol(symbol):
    # Remove quote currencies
    base_symbol = symbol.replace("USDT", "").replace("BUSD", "").replace("USD", "")

    try:
        url = "https://api.coingecko.com/api/v3/coins/list"
        response = requests.get(url, timeout=10)
        coins = response.json()

        for coin in coins:
            if coin["symbol"].lower() == base_symbol.lower():
                name = coin["name"]
                return [
                    name,
                    base_symbol.upper(),
                    f"{base_symbol.upper()}/USD",
                    f"{name} price",
                    f"{name} news"
                ]
    except Exception as e:
        print(f"⚠️ API Error for {symbol}: {e}")

    # If no match found
    return [
        base_symbol.upper(),
        f"{base_symbol.upper()}/USD",
        "cryptocurrency"
    ]

def enrich_keywords(keywords):
    return keywords + COMMON_BUZZWORDS


In [18]:
keywords_btc = generate_keywords_for_symbol("ETH")
print(keywords_btc)

# Output:
# ['Bitcoin', 'BTC', 'BTC/USD', 'Bitcoin price', 'Bitcoin news']


['Bifrost Bridged ETH (Bifrost)', 'ETH', 'ETH/USD', 'Bifrost Bridged ETH (Bifrost) price', 'Bifrost Bridged ETH (Bifrost) news']


In [1]:
from influxdb_client import InfluxDBClient

# Replace with your actual credentials
INFLUX_URL = "http://localhost:8086"
INFLUX_TOKEN = "ulx8xRTPRxKpGKTf0Vl_nfioXzS7vunTOxms74HY4tXXHrfPvrgrVrmj48VCOAStqoXNVV3V2b-Bar7SR0ihUw=="
INFLUX_ORG = "Sentry"
INFLUX_BUCKET = "Sentry"

client = InfluxDBClient(url=INFLUX_URL, token=INFLUX_TOKEN, org=INFLUX_ORG)


In [ ]:
# --- Setup ---
import pandas as pd
import json
import os
from influxdb_client import InfluxDBClient, Point
from influxdb_client.client.write_api import SYNCHRONOUS
from datetime import datetime, timezone


# --- InfluxDB Connection ---
# Replace with your actual credentials
INFLUX_URL = "http://localhost:8086"
INFLUX_TOKEN = "ulx8xRTPRxKpGKTf0Vl_nfioXzS7vunTOxms74HY4tXXHrfPvrgrVrmj48VCOAStqoXNVV3V2b-Bar7SR0ihUw=="
INFLUX_ORG = "Sentry"
INFLUX_BUCKET = "Sentry"

client = InfluxDBClient(url=INFLUX_URL, token=INFLUX_TOKEN, org=INFLUX_ORG)
write_api = client.write_api(write_options=SYNCHRONOUS)
query_api = client.query_api()

# --- Load symbols.csv ---
symbols_df = pd.read_csv("symbols.csv")
symbols = symbols_df["symbol"].dropna().unique().tolist()
print(f"✅ Loaded {len(symbols)} symbols.")

# --- Fixed Buzzwords ---
FIXED_BUZZWORDS = ["ETF", "DeFi", "SEC", "Bullish", "Bearish", "Pump", "Crash", "Rally", "Halving"]

# --- Simple Keyword Generator ---
def generate_keywords(symbol):
    base = symbol.replace("USDT", "").replace("BUSD", "").replace("USD", "")
    return [base.upper(), f"{base.upper()}/USD", f"{base.capitalize()}", f"{base.capitalize()} news"]

def get_existing_symbols():
    flux_query = f'''
    from(bucket: "{INFLUX_BUCKET}")
    |> range(start: -90d)
    |> filter(fn: (r) => r["_measurement"] == "symbol_metadata")
    |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
    |> keep(columns: ["symbol"])
    |> group()
    |> distinct(column: "symbol")
    '''
    result = query_api.query_data_frame(flux_query)
    if not result.empty:
        return set(result["symbol"].dropna().unique())
    return set()

existing_symbols = get_existing_symbols()
print(f"✅ {len(existing_symbols)} symbols already exist in Influx.")

# --- Write metadata ---
new_symbols = set(symbols) - existing_symbols
print(f"🔄 Preparing to write {len(new_symbols)} new symbols.")

for symbol in new_symbols:
    base_keywords = generate_keywords_for_symbol(symbol)
    enriched_keywords = enrich_keywords(base_keywords)

    now = datetime.now(timezone.utc)  # <-- Move inside the loop

    point = (
    Point("symbol_metadata")  # <-- explicitly here
    .tag("symbol", symbol)
    .field("keywords", ",".join(enriched_keywords))
    .field("buzzwords", ",".join(COMMON_BUZZWORDS))
    .time(now)
)

    write_api.write(bucket=INFLUX_BUCKET, record=point)
    print(f"✅ Metadata written for {symbol} at {now}")



print("🎯 All new symbols pushed successfully!")


✅ Loaded 770 symbols.


KeyError: 'symbol'

In [14]:
from influxdb_client import Point
from datetime import datetime, timezone

test_point = (
    Point("symbol_metadata")
    .tag("symbol", "TESTCOIN")
    .field("keywords", "Test,Example,Coin")
    .field("buzzwords", "TestBuzz")
    .time(datetime.now(timezone.utc))
)

try:
    write_api.write(bucket=INFLUX_BUCKET, record=test_point)
    print("✅ Test write succeeded!")
except Exception as e:
    print(f"❌ Write failed: {e}")


✅ Test write succeeded!


In [20]:
query = f'''
import "influxdata/influxdb/schema"
from(bucket: "{INFLUX_BUCKET}")
|> range(start: -7d)
|> filter(fn: (r) => r["_measurement"] == "symbol_metadata")
|> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
'''

df = query_api.query_data_frame(query)
print(df)

    result  table                           _start  \
0  _result      0 2025-04-19 21:34:46.276356+00:00   

                             _stop                            _time  \
0 2025-04-26 21:34:46.276356+00:00 2025-04-26 21:32:03.510226+00:00   

      _measurement    symbol buzzwords           keywords  
0  symbol_metadata  TESTCOIN  TestBuzz  Test,Example,Coin  
